In [ ]:
import osimport syssys.path.insert(0, os.path.dirname(os.getcwd()))  # For Notebookfrom firewall_api import Firewall, LIKE, EQ, NOTimport jsonimport ipaddressimport csv# Load Firewall Credentialswith open("../Credentials/firewall_access.json", "r") as file:    firewall_info = json.load(file)firewall = Firewall(    username=firewall_info["username"],    password=firewall_info["password"],    hostname=firewall_info["firewall_ip"],    port=firewall_info["port"],    certificate_verify=False,)# Test connectionprint(f"Connection Test ::  {firewall.read("Login")}\n")# Load objects from CSVwith open("objects.csv", "r", encoding="UTF8", newline="") as csv_file:    list_of_objects = list(csv.DictReader(csv_file))# Common constantsPREFIX = "MSS_"COMMON_DESCRIPTION = "Created by Integrity360 Sophos Firewall API"# Function to create and log entitiesdef create_entity(firewall, entity_type, entity_data, print_data=False, print_result=False):    if print_data:        print(f"Creating {entity_type} with data: {entity_data}")    response = firewall.create(entity_type, entity_data)    if print_result:        print(f"CREATE :: {response}\n")    return response# Create IPHostGroupip_host_group_name = f"{PREFIX}Brutal Force Group"create_entity(    firewall,    "IPHostGroup",    {        "Name": ip_host_group_name,        "IPFamily": "IPv4",        "Description": COMMON_DESCRIPTION,    },)# Create FQDNHostGroupfqdn_group_name = f"{PREFIX}FQDN_Group"create_entity(    firewall,    "FQDNHostGroup",    {        "Name": fqdn_group_name,        "IPFamily": "IPv4",        "Description": COMMON_DESCRIPTION,    },)# Process each object from the CSVfor item in list_of_objects:    entity_type = None    entity_data = {}    if item["type"] == "ipv4":        entity_type = "IPHost"        entity_data["IPFamily"] = "IPv4"        if "-" in item["data"]:  # IP range            start_ip, end_ip = item["data"].replace(" ", "").split("-")            entity_data.update(                {                    "Name": f"{PREFIX}IPR_{item['data']}",                    "HostType": "IPRange",                    "StartIPAddress": start_ip,                    "EndIPAddress": end_ip,                }            )        else:  # Single IP or Network            network = ipaddress.IPv4Network(item["data"])            if str(network.netmask) == "255.255.255.255":  # Single IP                entity_data.update(                    {                        "Name": f"{PREFIX}IPH_{item['data']}",                        "HostType": "IP",                        "IPAddress": str(network.network_address),                    }                )            else:  # Network                entity_data.update(                    {                        "Name": f"{PREFIX}IPNW_{item['data']}",                        "HostType": "Network",                        "IPAddress": str(network.network_address),                        "Subnet": str(network.netmask),                    }                )        entity_data["HostGroupList"] = {"HostGroup": ip_host_group_name}    elif item["type"] == "fqdn":        entity_type = "FQDNHost"        entity_data.update(            {                "Name": f"{PREFIX}fqdn_{item['data']}",                "FQDN": item["data"],                "FQDNHostGroupList": {"FQDNHostGroup": [fqdn_group_name]},            }        )    else:        print(f"Unsupported type: {item['type']} - Skipping item.")        continue  # Skip unsupported types    # Add common properties and create entity    entity_data["Description"] = COMMON_DESCRIPTION    create_entity(firewall, entity_type, entity_data)

In [ ]:
##   DELETE## firewall = Firewall(username, password, firewall_ip, port, certificate_verify, password_encrypted)entity_name = PREFIXentity_type = "IPHostGroup"print("DELETE :: ", firewall.delete(entity_type, entity_name, LIKE))entity_type = "IPHost"print("DELETE :: ", firewall.delete(entity_type, entity_name, LIKE))entity_type = "FQDNHostGroup"print("DELETE :: ", firewall.delete(entity_type, entity_name, LIKE))entity_type = "FQDNHost"print("DELETE :: ", firewall.delete(entity_type, entity_name, LIKE))